This is the solution to Fervo Energy Interview video timeseries problem

In [1]:
#!sudo apt -q install tesseract-ocr
!pip -q install pytesseract

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import cv2
import os
import pytesseract
from datetime import datetime
import argparse
import base64
import multiprocessing
from functools import partial
import time
import boto3
import struct
import pickle
import json
import botocore

In [ ]:
lambda_name        = "FervoEnergyVideo-VideoProcessing-Nv5REvJ0kicr"             # Name of the Lambda function to invoke
results_bucket     = "video-results-obinnak"                    # Bucket to use
results_folder    = "cpu_df"  # Subfolder to place the calculation results
concurrent_lambdas = 100

source_bucket   = 'obinnak-public'    # S3 bucket name with input data
source_folder   = 'data'         # Folder path, sometimes referred to as the prefix
source_filename = 'Test_Video.mp4' # Filename
is_gpu          = 'False'

In [ ]:
# Get S3 object
# Get the file stream
s3 = boto3.resource('s3')
data_obj = s3.Object(source_bucket, f"{source_folder}/{source_filename}")

# Define some needed variables based off the above parameters
start_time = time.time()

results_file_list = [] # Lets keep track of the output file names, so we can grab them later

lambda_client = boto3.client('lambda')

# Build the message for the Lambda to find the video file
payload = {
    "bucket_in"          : source_bucket,
    "folder_in"          : source_folder,
    "filename_in"        : source_filename,
    "bucket_out"         : results_bucket,
    "folder_out"         : results_folder,
    "is_gpu"             : is_gpu
    }

# Invoke the Lambda SegyBatchProcessMeanAmp
response = lambda_client.invoke(FunctionName=lambda_name,
                                InvocationType='Event',
                                Payload=json.dumps(payload))

results_file_list.append(f"{results_folder}/{source_filename}.pkl")

print("Done!  Elapse time to gather send to and processing on Lambda: {:0.2f} seconds.  Now waiting for 80 seconds.".format(time.time() - start_time))

time.sleep(80)      # Waiting before carrying on next steps, to allow time for Lambda to finish.

In [ ]:
s3_client = boto3.client('s3')
# Get file from S3, convert from Pickle format
object = s3_client.get_object(Bucket=results_bucket, Key=results_file_list[x])

serializedObject = object['Body'].read()
results_bundle_temp = pickle.loads(serializedObject)